In [ ]:
# importing required packages
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
# from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import cohen_kappa_score
from keras.layers import TimeDistributed
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten,Bidirectional
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

In [ ]:
import csv

In [ ]:
importing_dataset = pd.read_csv('https://github.com/dnyanada02/SmartGrading/blob/main/Dataset/training_set_rel3.tsv?raw=true', quoting=csv.QUOTE_NONE, sep='\t', encoding='ISO-8859-1')
# dependent variable
scores = importing_dataset['domain1_score']
dataset = importing_dataset.loc[:,['essay_id', 'essay_set', 'essay', 'domain1_score']]
dataset.dropna()
dataset

# dataset = pd.read_csv("/content/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1',
#                             usecols = ['essay_id', 'essay_set', 'essay','domain1_score']).dropna(axis=1)
# scores = dataset['domain1_score']
# dataset

,essay_id,essay_set,essay,domain1_score
0,1,1,"""Dear local newspaper, I think effects compute...",8
1,2,1,"""Dear @CAPS1 @CAPS2, I believe that using comp...",9
2,3,1,"""Dear, @CAPS1 @CAPS2 @CAPS3 More and more peop...",7
3,4,1,"""Dear Local Newspaper, @CAPS1 I have found tha...",10
4,5,1,"""Dear @LOCATION1, I know having computers has ...",8
...,...,...,...,...
12973,21626,8,""" In most stories mothers and daughters are ei...",35
12974,21628,8,""" I never understood the meaning laughter is t...",32
12975,21629,8,"""When you laugh, is @CAPS5 out of habit, or is...",40
12976,21630,8,""" Trippin' on fe...",40


In [ ]:
dataset.describe()

,essay_id,essay_set,domain1_score
count,12978.000000,12978.000000,12978.000000
mean,10295.432809,4.179458,6.799276
std,6308.588616,2.136749,8.970357
min,1.000000,1.000000,0.000000
25%,4439.250000,2.000000,2.000000
50%,10045.500000,4.000000,3.000000
75%,15680.750000,6.000000,8.000000
max,21633.000000,8.000000,60.000000


In [ ]:
# Generating word tokens after removing characters other than alphabets, converting them to lower case and
# removing stopwords from the text'''

def word_tokens(essay_text):
    essay_text = re.sub("[^a-zA-Z]", " ", essay_text)
    words = essay_text.lower().split()
    stop_words = set(stopwords.words("english"))
    words = [w for w in words if not w in stop_words]
    return (words)

In [ ]:
# Generating sentence tokens from the essay and finally the word tokens

def sentence_tokens(essay_text):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_tokens = tokenizer.tokenize(essay_text.strip())
    sentences = []
    for sent_token in sent_tokens:
        if len(sent_token) > 0:
            sentences.append(word_tokens(sent_token))
    return sentences

In [ ]:
# Generating a vector of features

def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

In [ ]:
# Generating word vectors to be used in word2vec model

def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay_text in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay_text, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [ ]:
from keras.layers import *
from keras.models import *
from keras import backend as K

In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [ ]:
def create_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=[1, 300])
    LSTM_layer = LSTM(300, dropout=0.2, recurrent_dropout=0.2, input_shape=[1, 300], return_sequences=True)(x)
    LSTM_layer = Bidirectional(LSTM(64, return_sequences=True))(LSTM_layer)
    attention_layer = attention()(LSTM_layer)
    # LSTM_layer = LSTM(300, dropout=0.2, recurrent_dropout=0.2, input_shape=[1, 300], return_sequences=True)(attention_layer)
    # attention_layer2 = attention()(attention_layer)
    # outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    outputs=Dense(1, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='mse', optimizer='adam',metrics=['mae'])    
    return model

In [ ]:
hidden_units = 2

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Applying k-fold cross validation

cv = KFold(n_splits=5, shuffle=True)
cv.get_n_splits(len(dataset))
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(dataset):
    print("\n------------Fold {}------------\n".format(count))
    X_test, X_train, y_test, y_train = dataset.iloc[testcv], dataset.iloc[traincv], scores.iloc[testcv], scores.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training set of essays.
            sentences += sentence_tokens(essay)
            
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_text in train_essays:
        clean_train_essays.append(word_tokens(essay_text))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_text in test_essays:
        clean_test_essays.append(word_tokens(essay_text))
    testDataVecs = getAvgFeatureVecs(clean_test_essays, model, num_features)
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    trainDataVecs = np.nan_to_num(trainDataVecs.astype(np.float32))
    testDataVecs = np.nan_to_num(testDataVecs.astype(np.float32))

    # lstm_model = get_model()
    # lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')


    lstm_model= create_with_attention(hidden_units=hidden_units, dense_units=64, input_shape=(300,1), activation='relu')
    lstm_model.summary() 
    history = lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    lstm_model.save("Attention_LSTM.h5")
    y_pred = lstm_model.predict(testDataVecs)
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    '''Evaluation metric used : 
    1. Mean squared error
    2. Variance
    3. Cohen's kappa score
    Expected results - Minimum error, maximum variance(For variance, best possible score is 1.0, lower 
    values are worse.) and maximum kappa score(1 depicting the best scores)'''
    
    # Mean squared error
    print("Mean squared error: {0:.2f}".format(mean_squared_error(y_test.values, y_pred)))

    # Explained variance score: 1 is perfect prediction
    print('Variance: {0:.2f}'.format(explained_variance_score(y_test.values, y_pred)))  
    
    #Cohen's kappa score
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {0:.2f}".format(result))
    results.append(result)

    count += 1


------------Fold 1------------

Training Word2Vec Model...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 lstm_10 (LSTM)              (None, 1, 300)            721200    
                                                                 
 bidirectional_5 (Bidirectio  (None, 1, 128)           186880    
 nal)                                                            
                                                                 
 attention_5 (attention)     (None, 128)               129       
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 908,338
Trainable params: 908,338
Non-trainable params: 0
_____________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 lstm_12 (LSTM)              (None, 1, 300)            721200    
                                                                 
 bidirectional_6 (Bidirectio  (None, 1, 128)           186880    
 nal)                                                            
                                                                 
 attention_6 (attention)     (None, 128)               129       
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 908,338
Trainable params: 908,338
Non-trainable params: 0
_____________________________________________________

...

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 lstm_14 (LSTM)              (None, 1, 300)            721200    
                                                                 
 bidirectional_7 (Bidirectio  (None, 1, 128)           186880    
 nal)                                                            
                                                                 
 attention_7 (attention)     (None, 128)               129       
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 908,338
Trainable params: 908,338
Non-trainable params: 0
_____________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 lstm_16 (LSTM)              (None, 1, 300)            721200    
                                                                 
 bidirectional_8 (Bidirectio  (None, 1, 128)           186880    
 nal)                                                            
                                                                 
 attention_8 (attention)     (None, 128)               129       
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 908,338
Trainable params: 908,338
Non-trainable params: 0
_____________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1, 300)]          0         
                                                                 
 lstm_18 (LSTM)              (None, 1, 300)            721200    
                                                                 
 bidirectional_9 (Bidirectio  (None, 1, 128)           186880    
 nal)                                                            
                                                                 
 attention_9 (attention)     (None, 128)               129       
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 908,338
Trainable params: 908,338
Non-trainable params: 0
_____________________________________________________

In [ ]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=2))

Average Kappa score after a 5-fold cross validation:  0.96


# Prediction

In [ ]:
# As lstm outperforms all other models, so using it for predicting the scores for the final dataset
valid_set = pd.read_csv('https://github.com/dnyanada02/SmartGrading/blob/main/Dataset/valid_set.tsv?raw=true', sep='\t', encoding='ISO-8859-1')

In [ ]:
valid_set = valid_set.drop(['domain2_predictionid'], axis = 1)

In [ ]:
valid_set.head()

,essay_id,essay_set,essay,domain1_predictionid
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792


In [ ]:
valid_test_essays = valid_set['essay']

In [ ]:
valid_test_essays

0       Dear @ORGANIZATION1, @CAPS1 more and more peop...
1       Dear @LOCATION1 Time @CAPS1 me tell you what I...
2       Dear Local newspaper, Have you been spending a...
3       Dear Readers, @CAPS1 you imagine how life woul...
4       Dear newspaper, I strongly believe that comput...
                              ...                        
4213     Have you ever noticed that if two little kids...
4214                                Laughter @CAPS1 I ...
4215     Laughter in @CAPS1 A laugh is not just an act...
4216      LAUGHTER @CAPS1 i was younger my friend live...
4217     You know how the saying goes live, laugh, lov...
Name: essay, Length: 4218, dtype: object

In [ ]:
sentences = []
    
for valid_essay in valid_test_essays:
        sentences += sentence_tokens(valid_essay)
            
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

valid_clean_test_essays = []
    
# Generate training and testing data word vectors.
for essay_text in valid_test_essays:
    valid_clean_test_essays.append(word_tokens(essay_text))
valid_testDataVecs = getAvgFeatureVecs(valid_clean_test_essays, model, num_features)

valid_testDataVecs = np.array(valid_testDataVecs)
# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
valid_testDataVecs = np.reshape(valid_testDataVecs, (valid_testDataVecs.shape[0], 1, valid_testDataVecs.shape[1]))
    
predicted_scores = lstm_model.predict(valid_testDataVecs)
    
# Round y_pred to the nearest integer.
predicted_scores = np.around(predicted_scores)

Training Word2Vec Model...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
submission = valid_set.drop(['essay'], axis = 1)

In [ ]:
predicted_score = predicted_scores.tolist()

In [ ]:
predicted_score = pd.Series([score for sublist in predicted_scores for score in sublist])

In [ ]:
predicted_score

0        9.0
1        6.0
2       11.0
3       12.0
4        9.0
        ... 
4213    34.0
4214    17.0
4215    26.0
4216    10.0
4217    20.0
Length: 4218, dtype: float32

In [ ]:
submission = pd.concat([submission, predicted_score], axis = 1).rename(columns = {0:"predicted_score"}).iloc[:,[2,0,1,3]]
submission.to_excel("Submission.xls",index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,cohen_kappa_score

# print('training accuracy:',trainDataVecs[1]*100)
# print('testing accuracy:',testDataVecs[1]*100)

# y_pred = lstm_model.predict(testDataVecs)
# y_pred = np.around(y_pred)
print(classification_report(y_test.values,y_pred))
print('Confusion matix:\n',confusion_matrix(y_test.values,y_pred))
print('Cohen-kappa score:',cohen_kappa_score(y_test.values,y_pred,weights='quadratic'))

              precision    recall  f1-score   support

         0.0       0.21      0.07      0.11        81
         1.0       0.54      0.41      0.46       348
         2.0       0.41      0.56      0.47       469
         3.0       0.49      0.61      0.54       588
         4.0       0.59      0.23      0.33       311
         5.0       0.09      0.05      0.06        20
         6.0       0.14      0.04      0.06        26
         7.0       0.10      0.14      0.12        22
         8.0       0.44      0.40      0.42       130
         9.0       0.20      0.47      0.28        73
        10.0       0.38      0.15      0.21        82
        11.0       0.08      0.03      0.04        33
        12.0       0.33      0.12      0.18        24
        13.0       0.19      0.19      0.19        16
        14.0       0.18      0.12      0.15        16
        15.0       0.10      0.20      0.14        15
        16.0       0.16      0.13      0.14        39
        17.0       0.17    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1